In [ ]:
from img_policy_contig_header import *

LOGGER = logging.getLogger()
LOGGER.setLevel(logging.INFO)
LOGGER.addHandler(logging.StreamHandler(sys.stdout))

# patch = np.asarray(Image.open(SAMPLE_IMG_PATH))
# pxImgHeight, pxImgWidth = image.shape[:2]  # (H,W,C)

In [ ]:
def memset(dst: List[int], src: List[int], sz: int):
    for i in range(sz):
        dst[i] = src[i]


FC_BUFFERS: List[List[int]] = [[] for _ in range(NUM_FLASH_CHANNELS)]


def get_fc_buffer_size() -> int:
    sz = len(FC_BUFFERS[0])
    for iFC in range(NUM_FLASH_CHANNELS):
        assert (sz == len(FC_BUFFERS[iFC])), "All FC_BUFFERS size should be the same"
    return sz


def _flush_page(iCh: int, data: List[int], sz: int):

    filename = f"logs/buffer.{iCh}.log"

    if sz != BYTES_PER_PAGE:
        raise RuntimeError("Unexpected data size")

    with open(filename, 'a') as f:
        text = ' '.join(map(str, data)) + '\n'
        f.write(text)

# end

In [ ]:
def flush_buffers(buffers: List[List[int]], bytesBuffered: int,  force: bool) -> int:

    global FC_BUFFERS

    bytesFlushed = 0
    while bytesBuffered >= BYTES_PER_PAGE:

        # flush channel buffers in a RR-liked policy
        for iCh in range(NUM_FLASH_CHANNELS):
            _flush_page(iCh, buffers[iCh][bytesFlushed:bytesFlushed+BYTES_PER_PAGE], BYTES_PER_PAGE)

        bytesBuffered -= BYTES_PER_PAGE
        bytesFlushed += BYTES_PER_PAGE

    # if force flush is on, padding zeros and do flush
    if force:
        bytesPadding = BYTES_PER_PAGE - bytesBuffered
        bytesBuffered = 0

        for iCh in range(NUM_FLASH_CHANNELS):
            for iByte in range(bytesPadding):
                buffers[iCh].append(PADDING_DONT_CARE)

            _flush_page(iCh, buffers[iCh][bytesFlushed:bytesFlushed+BYTES_PER_PAGE], BYTES_PER_PAGE)

    else:
        for iFC in range(NUM_FLASH_CHANNELS):
            FC_BUFFERS[iFC] = buffers[iFC][bytesFlushed:bytesFlushed+bytesBuffered]

        LOGGER.debug("Buffer not full and force flush is disabled")

        if bytesBuffered != get_fc_buffer_size():
            raise RuntimeError(f"bytesBuffered {bytesBuffered} != FC_BUFFER's size {get_fc_buffer_size()}")

    return bytesBuffered

# end

In [ ]:
def block_dispatcher(rows: List[int], pxWidth: int):

    assert (pxWidth % 4 == 0), f"The width of block row should be 4N, but got {pxWidth}"

    # read block by block
    global FC_BUFFERS
    iByte = 0
    for iPx in range(0, pxWidth, PX_BLK_WIDTH):

        # dispatch chunks (w/ RGB channels) to FC_BUFFERS
        for iRow in range(PX_BLK_HEIGHT):

            FC_BUFFERS[iRow].append(rows[iRow][iByte+0])  # R0
            FC_BUFFERS[iRow].append(rows[iRow][iByte+3])  # R1
            FC_BUFFERS[iRow].append(rows[iRow][iByte+1])  # G0
            FC_BUFFERS[iRow].append(rows[iRow][iByte+4])  # G1
            FC_BUFFERS[iRow].append(rows[iRow][iByte+2])  # B0
            FC_BUFFERS[iRow].append(rows[iRow][iByte+5])  # B1

            FC_BUFFERS[PX_BLK_HEIGHT+iRow].append(rows[iRow][iByte+0+6])  # R2
            FC_BUFFERS[PX_BLK_HEIGHT+iRow].append(rows[iRow][iByte+3+6])  # R3
            FC_BUFFERS[PX_BLK_HEIGHT+iRow].append(rows[iRow][iByte+1+6])  # G2
            FC_BUFFERS[PX_BLK_HEIGHT+iRow].append(rows[iRow][iByte+4+6])  # G3
            FC_BUFFERS[PX_BLK_HEIGHT+iRow].append(rows[iRow][iByte+2+6])  # B2
            FC_BUFFERS[PX_BLK_HEIGHT+iRow].append(rows[iRow][iByte+5+6])  # B3

        LOGGER.debug(f"block dispatched: \n{rows[:,iByte:iByte+12]}\n")
        LOGGER.debug("current FC_BUFFERS content:")
        for iFC in range(NUM_FLASH_CHANNELS):
            LOGGER.debug(FC_BUFFERS[iFC])
        LOGGER.debug("\n")

        iByte += PX_BLK_WIDTH * BYTES_PER_PIXEL

    bytesBuffered = flush_buffers(FC_BUFFERS, get_fc_buffer_size(), False)
    FC_BUFFERS = [FC_BUFFERS[iFC][:bytesBuffered] for iFC in range(NUM_FLASH_CHANNELS)]

    LOGGER.debug(f"Truncate FC_BUFFERS to {bytesBuffered} bytes")

# end

In [ ]:
def patch_dispatcher(iPatch: int, patch: np.ndarray):

    assert (patch.shape[1] % BYTES_PER_PIXEL == 0), "the patch should already be rearranged in the caller function"
    try:
        # original patch size info
        pxPatchHeight, bytesPatchWidth = patch.shape[:2]  # (H,W)
        pxPatchWidth = bytesPatchWidth // 3  # RGB 3 channels

        # make sure width and height are 4N
        pxBufHeight = pxPatchHeight + (0 if pxPatchHeight % 4 == 0 else 4 - (pxPatchHeight % 4))
        pxBufWidth = pxPatchWidth + (0 if pxPatchWidth % 4 == 0 else 4 - (pxPatchWidth % 4))
        bytesBufWidth = pxBufWidth * 3

        assert ((pxBufHeight <= PX_PATCH_HEIGHT) and (pxBufHeight % 4 == 0)), "Unexpected Patch Buffer Size"
        assert ((pxBufWidth <= PX_PATCH_WIDTH) and (pxBufWidth % 4 == 0)), "Unexpected Patch Buffer Size"

        if (pxBufHeight != PX_PATCH_HEIGHT) or (pxBufWidth != PX_PATCH_WIDTH):
            LOGGER.info(f"Patch[{iPatch}] is Partial Patch, shape = ({pxBufHeight},{pxBufWidth})")
        else:
            LOGGER.debug(f"Patch[{iPatch}] is Full Patch")

        # flatten patch array (as 1D array)
        patch = patch.reshape(-1)

        # handle patch rows
        iBufRow = 0
        bytesHandled = 0
        blkBuffers = np.full((4, bytesBufWidth), PADDING_DONT_CARE, dtype=np.uint8)
        for iPatchRow in range(0, pxPatchHeight):

            # copy a whole row to buffer
            memset(blkBuffers[iBufRow][:], patch[bytesHandled:bytesHandled+bytesPatchWidth], bytesPatchWidth)

            # update counters of next row and data offset
            iBufRow += 1
            bytesHandled += bytesPatchWidth

            # once buffer full (4 rows), reset counter and do dispatching
            if iBufRow == 4:
                iBufRow = 0
                block_dispatcher(blkBuffers, pxBufWidth)

                # reset blkBuffers
                blkBuffers = np.full((4, bytesBufWidth), PADDING_DONT_CARE, dtype=np.uint8)

        # if some data not flushed yet
        if iBufRow > 0:
            # padding remaining block buffer rows
            for i in range(4 - iBufRow):
                blkBuffers[iBufRow][:] = np.full(bytesBufWidth, PADDING_DONT_CARE, dtype=np.uint8)

            # force flush remaining img and padding data
            block_dispatcher(blkBuffers, pxBufWidth)

    except Exception as ex:
        pp(ex)
        raise ex

In [ ]:
def image_dispatcher(image: np.ndarray):

    global FC_BUFFERS

    try:
        pxImgHeight, pxImgWidth = image.shape[:2]  # (H,W,C)
        bytesImgWidth = pxImgWidth * BYTES_PER_PIXEL  # RGB 3 channels

        # allocate buffers for PATCH_HEIGHT image rows (row majoy patch by patch)
        pxBufHeight = PX_PATCH_HEIGHT
        pxBufWidth = pxImgWidth  # zeros may need to be padded to the image
        bytesBufWidth = pxBufWidth * BYTES_PER_PIXEL

        # flatten image array (as 1D array)
        image = image.reshape(-1)

        # handle image patch by patch (row major)
        iBufRow = 0
        bytesHandled = 0
        numPatchesHandled = 0
        rowBuffers = np.full((pxBufHeight, bytesBufWidth), PADDING_DONT_CARE, dtype=np.uint8)

        for iImgRow in range(pxImgHeight):
            # copy row to buffer
            memset(rowBuffers[iBufRow][:], image[bytesHandled:bytesHandled+bytesImgWidth], bytesImgWidth)

            # update counters
            iBufRow += 1
            bytesHandled += bytesImgWidth

            # once buffer full (4 rows), reset counter and do dispatching
            if iBufRow == pxBufHeight:

                iBufRow = 0

                nFullPatches = pxBufWidth // PX_PATCH_WIDTH
                pxPartialPatchWidth = pxBufWidth % PX_PATCH_WIDTH

                # dispatch patch by patch
                off = 0
                for iPatch in range(nFullPatches):
                    patch_dispatcher(numPatchesHandled, rowBuffers[:, off:off+BYTES_PATCH_WIDTH])
                    off += BYTES_PATCH_WIDTH

                    numPatchesHandled += 1

                # partital patch (full height)
                if pxPartialPatchWidth > 0:
                    patch_dispatcher(numPatchesHandled, rowBuffers[:, off:])
                    numPatchesHandled += 1

        # handle remaining rows (last patch row, partial patch)
        if iBufRow > 0:
            nFullWidthPatches = pxBufWidth // PX_PATCH_WIDTH
            pxPartialPatchWidth = pxBufWidth % PX_PATCH_WIDTH

            # dispatch patch by patch
            off = 0
            for iPatch in range(nFullWidthPatches):
                patch_dispatcher(numPatchesHandled, rowBuffers[:iBufRow, off:off+BYTES_PATCH_WIDTH])
                numPatchesHandled += 1
                off += BYTES_PATCH_WIDTH

            # partital patch (full height)
            if pxPartialPatchWidth > 0:
                patch_dispatcher(numPatchesHandled, rowBuffers[:iBufRow, off:])
                numPatchesHandled += 1

        # if buffer still not empty
        if get_fc_buffer_size() > 0:
            LOGGER.info("Some data still in FC_BUFFERS but the image dispatching is finished , do force flush")
            flush_buffers(FC_BUFFERS, get_fc_buffer_size(), True)

    except Exception as ex:
        pp(ex)
        raise ex

# end

In [ ]:
# image = np.asarray(Image.open(SAMPLE_IMG_PATH))
image = np.random.randint(0, 256, (7891, 6542, 3), dtype=np.uint8)

raise RuntimeError()

In [ ]:
image = np.load("./verify.npy")

# clear file content
for iFC in range(NUM_FLASH_CHANNELS):
    open(f"logs/buffer.{iFC}.log", "w").close()


image_dispatcher(image)

In [ ]:
raise RuntimeError()

In [ ]:
IMG_PATH = "./random.tiff"

image = np.random.randint(0, 256, (10000, 10000, 3), dtype=np.uint8)
pImg = Image.fromarray(image)
pImg.save(IMG_PATH, compression='none')


In [ ]:
from img_policy_contig_verify import *

# image = np.load("./verify.npy")
image = np.asarray(Image.open("./random.tiff"))
verify_image(image, False)